# 💬 Chat-Commerce 🛒

## Setup

In [ ]:
# Install packages
%pip install python-dotenv langchain chromadb

In [1]:
from dotenv import load_dotenv

# Load environment values
load_dotenv()

True

In [2]:
import logging
import sys

# Turn off unwanted logs
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

## Index E-Commerce Data

In [3]:
from langchain.document_loaders import CSVLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

loader = CSVLoader('./data/home-and-garden.csv', csv_args={
    
})
documents = loader.load()

embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(documents, embeddings, persist_directory="./index/")
docsearch.persist()

Running Chroma using direct local API.
DEBUG:Chroma:Logger created
Logger created
INFO:clickhouse_connect.driver.ctypes:Successfully imported ClickHouse Connect C data optimizations
Successfully imported ClickHouse Connect C data optimizations
INFO:clickhouse_connect.driver.ctypes:Successfully import ClickHouse Connect C/Numpy optimizations
Successfully import ClickHouse Connect C/Numpy optimizations
INFO:clickhouse_connect.json_impl:Using python library for writing JSON byte strings
Using python library for writing JSON byte strings
loaded in 80 embeddings
loaded in 1 collections
collection with name langchain already exists, returning existing collection
DEBUG:Chroma:Index saved to ./index//index/index.bin
Index saved to ./index//index/index.bin
Persisting DB to disk, putting it in the save folder ./index/


## Run Queries

In [4]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(temperature=0)

In [5]:
from langchain import PromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

system_prompt = PromptTemplate(
    template="""
You are a shopping assistant that helps visitors find the right product from the store.
If you can't find the answer from the context, ask more question. Do not make
Once you have an answer, list down the products in a clear tone as markdown list.
----------------
{context}""",
    input_variables=["context"],
)

system_message_prompt = SystemMessagePromptTemplate(prompt=system_prompt)

human_message_prompt = HumanMessagePromptTemplate.from_template("{question}")

chat_prompt = ChatPromptTemplate.from_messages(
    [system_message_prompt, human_message_prompt],
)


In [6]:
from langchain.chains import ChatVectorDBChain

qa = ChatVectorDBChain.from_llm(
    llm=chat,
    vectorstore=docsearch,
    qa_prompt=chat_prompt,
)


In [7]:
import IPython

chat_history = []

def ask(query):
    result = qa({"question": query, "chat_history": chat_history})
    chat_history.append((query, result["answer"]))
    IPython.display.display(IPython.display.Markdown(result["answer"]))


In [8]:
ask("Hey! Can you help me find something for my garden?")

DEBUG:Chroma:time to pre process our knn query: 1.1920928955078125e-06
time to pre process our knn query: 1.1920928955078125e-06
DEBUG:Chroma:time to run knn query: 0.00046062469482421875
time to run knn query: 0.00046062469482421875


Of course! What specifically are you looking for? Do you need tools, decorations, or plants?

In [9]:
ask("I'm looking for pots")

DEBUG:Chroma:time to pre process our knn query: 3.0994415283203125e-06
time to pre process our knn query: 3.0994415283203125e-06
DEBUG:Chroma:time to run knn query: 0.0005080699920654297
time to run knn query: 0.0005080699920654297


I'm not looking for any specific pots, I just provided the information for a white ceramic pot. Is there a particular type of pot you are interested in?

In [10]:
ask("Suggest me a pot which is environmentally friendly")

DEBUG:Chroma:time to pre process our knn query: 2.1457672119140625e-06
time to pre process our knn query: 2.1457672119140625e-06
DEBUG:Chroma:time to run knn query: 0.000492095947265625
time to run knn query: 0.000492095947265625


Sure! We have biodegradable cardboard pots from Rustic LTD that are perfect for outdoor gardening. They are eco-friendly and will naturally decompose over time, making them a great choice for the environment.

In [11]:
ask("yes")

DEBUG:Chroma:time to pre process our knn query: 5.0067901611328125e-06
time to pre process our knn query: 5.0067901611328125e-06
DEBUG:Chroma:time to run knn query: 0.0005028247833251953
time to run knn query: 0.0005028247833251953


Sure, here are some other environmentally friendly pot options:

- Terracotta pots
- Ceramic pots
- Coconut coir pots
- Peat pots
- Fabric pots
- Glass pots
- Metal pots (such as copper or stainless steel)
- Wood pots (such as bamboo or cedar)